In [ ]:
# Setup the Jupyter version of Dash
from dash import Dash
from index import CRUD

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt




###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "root"
host = "nv-desktop-services.apporto.com"
port = 30393
database = "AAC"
collection = "animals"



# Connect to database via CRUD Module
db = CRUD(username, password, host, port, database, collection)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = Dash('My Dash-App')

# Adds a way to load our image for the logo
image_filename = 'logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Center(html.Img
                (src='data:image/png;base64,{}'.format(encoded_image.decode())
                , alt="logo", style={'max-width':'30%', 'max-height': '30%'})), 
# Adds a bit of a dyamic change when adjusting screen size / I thought the logo was taking to much space
    
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('CS-340 Dashboard Chritian Pagano'))),
    html.Hr(),
    
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'All', 'value': 'all'},
            {'label': 'Water Rescue', 'value': 'water_rescue'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
            {'label': 'Disaster Rescue or Individual Tracking', 'value': 'rescue'},
        ],
            value = 'all'

    ),
    
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         
    
        data=df.to_dict('records'),
        editable=True,
        row_selectable="single", #allow a row to be selected
        selected_rows=[],
        filter_action="native", #allow a filter
        sort_action="native", #allow sorting
        page_action="native", #enable pagination
        page_current=0, #set start page
        page_size=10, #set rows per page

                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id', 'columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    
    # created types with dogs information
    # sorted by
    # breed [],
    # gender [],
    # trainging age []
    water_type = [
        ['Labrador Retriever Mix',
        'Chesapeake Bay Retriever',
        'Newfoundland'
        ],
        ['Intact Female'],
        [26, 156]
        ]
              
    mountain_type = [
        ['German Shepherd', 
        'AlaskanMalamute', 
        'Old English Sheepdog', 
        'Siberian Husky',
        'Rottweiler'
        ],
        ['Intact Male'],
        [26, 156]
        ]
              
    tracking_type = [
        ['Doberman Pinscher', 
        'German Shepherd', 
        'Golden Retriever',
        'Bloodhound', 
        'Rottweiler'
        ],
        ['Intact Male'],
        [20, 300]
        ]
    
    # created an assocation between my types and my filter_type
    
    breed_type = {
        'water_rescue': water_type,
        'mountain': mountain_type,
        'rescue': tracking_type
    }
              
    if filter_type in breed_type:
        breed_data = breed_type[filter_type]
    elif filter_type == 'all':
        breed_data = None
    else:
        raise Exception('Unkown filter')
    
              
    if breed_data is None:
              query = {}
              
    # use regex to create a query to search for specific dogs, while also keeping a strong amount
    # of flexablity since the customer wants to be able to use this else were
    # typos may get left out but if it has a weird ending or begining it may be saved
    else:
        breed_names, gender, training_age = breed_data
        regex_search = [re.compile(f'.*{dog_breed}.*', re.IGNORECASE) for dog_breed in breed_names]
        query = {
            '$or': [{"breed": {'$regex': search}} for search in regex_search],
            'sex_upon_outcome': gender[0],
            'age_upon_outcome_in_weeks': {'$gte': training_age[0], '$lte': training_age[1]}
        }
    # get dogs from the shelter based on query
    df = pd.DataFrame.from_records(db.read(query))

    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data = df.to_dict('records')
    print(df)
    return (data, columns)
              

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    
    dffPie = pd.DataFrame.from_dict(viewData)
    
    return[
        dcc.Graph(
            figure = px.pie(dffPie, names='breed')
        )
    ]


    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '600px', 'height': '350px'}, center=[30.75,-97.48], zoom=8, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run()


Dash is running on http://127.0.0.1:15572/

Dash is running on http://127.0.0.1:15572/

Dash is running on http://127.0.0.1:15572/

Dash is running on http://127.0.0.1:15572/

Dash is running on http://127.0.0.1:15572/

Dash is running on http://127.0.0.1:15572/

Dash is running on http://127.0.0.1:15572/

Dash is running on http://127.0.0.1:15572/

Dash is running on http://127.0.0.1:15572/

Dash is running on http://127.0.0.1:15572/

Dash is running on http://127.0.0.1:15572/

Dash is running on http://127.0.0.1:15572/

Dash is running on http://127.0.0.1:15572/

Dash is running on http://127.0.0.1:15572/

Dash is running on http://127.0.0.1:15572/

Dash is running on http://127.0.0.1:15572/

Dash is running on http://127.0.0.1:15572/

Dash is running on http://127.0.0.1:15572/

Dash is running on http://127.0.0.1:15572/

Dash is running on http://127.0.0.1:15572/

Dash is running on http://127.0.0.1:15572/

Dash is running on http://127.0.0.1:15572/

Dash is running on http://127.0.

 * Running on http://127.0.0.1:15572/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Dec/2023 02:34:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Dec/2023 02:34:21] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [11/Dec/2023 02:34:21] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [11/Dec/2023 02:34:21] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 304 -
127.0.0.1 - - [11/Dec/2023 02:34:21] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 304 -
127.0.0.1 - - [11/Dec/2023 02:34:21] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/anaconda/lib/python3.9/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/anaconda/lib/python3.9/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/anaconda/lib/python3.9/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/anaconda/lib/python3.9/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/anaconda/lib/python3.9/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/anaconda/lib/python3.9/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/usr/local/anaconda/lib/python3.9/site-packages/dash/dash.py", line 1

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/anaconda/lib/python3.9/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/anaconda/lib/python3.9/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/anaconda/lib/python3.9/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/anaconda/lib/python3.9/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/anaconda/lib/python3.9/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/anaconda/lib/python3.9/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/usr/local/anaconda/lib/python3.9/site-packages/dash/dash.py", line 1

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/anaconda/lib/python3.9/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/anaconda/lib/python3.9/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/anaconda/lib/python3.9/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/anaconda/lib/python3.9/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/anaconda/lib/python3.9/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/anaconda/lib/python3.9/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/usr/local/anaconda/lib/python3.9/site-packages/dash/dash.py", line 1

...

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/anaconda/lib/python3.9/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/anaconda/lib/python3.9/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/anaconda/lib/python3.9/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/anaconda/lib/python3.9/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/anaconda/lib/python3.9/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/anaconda/lib/python3.9/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/usr/local/anaconda/lib/python3.9/site-packages/dash/dash.py", line 1

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/anaconda/lib/python3.9/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/anaconda/lib/python3.9/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/anaconda/lib/python3.9/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/anaconda/lib/python3.9/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/anaconda/lib/python3.9/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/anaconda/lib/python3.9/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/usr/local/anaconda/lib/python3.9/site-packages/dash/dash.py", line 1

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/anaconda/lib/python3.9/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/anaconda/lib/python3.9/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/anaconda/lib/python3.9/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/anaconda/lib/python3.9/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/anaconda/lib/python3.9/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/anaconda/lib/python3.9/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/usr/local/anaconda/lib/python3.9/site-packages/dash/dash.py", line 1

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/anaconda/lib/python3.9/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/anaconda/lib/python3.9/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/anaconda/lib/python3.9/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/anaconda/lib/python3.9/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/anaconda/lib/python3.9/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/anaconda/lib/python3.9/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/usr/local/anaconda/lib/python3.9/site-packages/dash/dash.py", line 1

127.0.0.1 - - [11/Dec/2023 03:05:12] "POST /_dash-update-component HTTP/1.1" 500 -
